<a href="https://colab.research.google.com/github/tosiki1202/GenAI-app/blob/main/20250924.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

hello


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# 使用したいモデルの名前を指定
model_name = "rinna/japanese-gpt2-small"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)